In [114]:
import cirq

# may need to include
import scipy 
from scipy.linalg import cossin
import math

print(
    "cirq = " + cirq.__version__,
    "scipy = " + scipy.__version__,
    sep='\n'
)
    

cirq = 0.10.0
scipy = 1.6.2


In [92]:
from typing import List, Tuple

import numpy as np


def matrix_to_sycamore_operations(
    target_qubits: List[cirq.GridQubit], matrix: np.ndarray
) -> Tuple[cirq.OP_TREE, List[cirq.GridQubit]]:
    """A method to convert a unitary matrix to a list of Sycamore operations.

    This method will return a list of `cirq.Operation`s using the qubits and (optionally) ancilla
    qubits to implement the unitary matrix `matrix` on the target qubits `qubits`.
    The operations are also supported by `cirq.google.gate_sets.SYC_GATESET`.

    Args:
        target_qubits: list of qubits the returned operations will act on. The qubit order defined by the list
            is assumed to be used by the operations to implement `matrix`.
        matrix: a matrix that is guaranteed to be unitary and of size (2**len(qs), 2**len(qs)).
    Returns:
        A tuple of operations and ancilla qubits allocated.
            Operations: In case the matrix is supported, a list of operations `ops` is returned.
                `ops` acts on `qs` qubits and for which `cirq.unitary(ops)` is equal to `matrix` up
                 to certain tolerance. In case the matrix is not supported, it might return NotImplemented to
                 reduce the noise in the judge output.
            Ancilla qubits: In case ancilla qubits are allocated a list of ancilla qubits. Otherwise
                an empty list.
        .
    """
    ## do the QSD decomposition
    OP = QSD(matrix)
    
    return OP, []


In [ ]:
def generate_Mk():

In [97]:
debug = True

# for (c -s s c) and (d_i)
def DiagonalDecomposition(qs: List[cirq.GridQubit], A: np.ndarray):
    OP = []
    
    return OP

# AB_i = V, D, W (D/2)
def Demultiplex(qs: List[cirq.GridQubit], A: np.ndarray):
    OP = []
    #  calculate the dimension
    D = A.shape[0]
    d = int(math.log(D, 2))
                
    U1 = A[np.ix_(range(0,D//2),range(0,D//2))]
    U2 = A[np.ix_(range(D//2,D),range(D//2, D))]
    #print(U1.shape, U2.shape)
    
    # V, D = cirq.linalg.unitary_eig()
    Diag, V = scipy.linalg.schur(U1@U2, output="complex")
    #print(Diag.shape, V.shape)
    
    V = V;
    Diag = np.sqrt(Diag);
    W = Diag@(V.conj().T)@U2;
    
    OP += QSD(qs[:-1], V);
    OP += []
    OP += QSD(qs[:-1], W);
    
    return OP 
    

# QSD ~= CSD + Demultiplex
def QSD(qs: List[cirq.GridQubit], A: np.ndarray):
    OP = []
    #  calculate the dimension
    D = A.shape[0]
    d = int(math.log(D, 2))
    
    # edge condition, use cirq ZYZ
    if (d==1):
        a, b, c = cirq.linalg.deconstruct_single_qubit_matrix_into_angles(A)
        return [cirq.qs[0]]
    
    # Decompose U to AB1, CS, AB2
    from scipy.linalg import cossin
    L, CS, R = cossin(A, D/2, D/2)
    
    OP += Demultiplex(qs, L)
    OP += []
    OP += Demultiplex(qs, R)
    
    return OP

In [96]:
QSD([1,2], I2)

[]

In [115]:
# I2 = np.eye(2**1)
# cirq.linalg.deconstruct_single_qubit_matrix_into_angles(I) #in matrix, return angles

In [116]:
#I2 = np.eye(2**2)
#cossin(I2, 2, 2)

In [117]:
# np.sqrt(I*4)

## The supported gates on these qubits will be:
- 1 qubit gates:
    - [X/Z/Y]PowGate: cirq.google.Sycamore.validate_operation(cirq.X(a))
    - PhasedXZGate: cirq.google.Sycamore.validate_operation(cirq.PhasedXZGate(x_exponent=1, z_exponent=1, axis_phase_exponent=1.2)(a))
- 2 qubit gates:
    - sqrt of ISWAP: cirq.google.Sycamore.validate_operation(cirq.ISWAP(a,b)**0.5)
    - SYC gate: cirq.google.Sycamore.validate_operation(cirq.google.SYC(a,b))

In [126]:
a, b = cirq.GridQubit(3,4), cirq.GridQubit(3,5)
print(
cirq.google.Sycamore.validate_operation(cirq.X(a)),
cirq.google.Sycamore.validate_operation(cirq.PhasedXZGate(x_exponent=1, z_exponent=1, axis_phase_exponent=1.2)(a)),
cirq.google.Sycamore.validate_operation(cirq.ISWAP(a,b)**0.5),
cirq.google.Sycamore.validate_operation(cirq.google.SYC(a,b)),
)

None None None None


In [129]:
from scipy.stats import unitary_group
x = unitary_group.rvs(4)

In [130]:
x

array([[ 0.13015817+0.73287303j, -0.54007754+0.21833927j,
        -0.25825206+0.08133333j, -0.02023084+0.18133301j],
       [-0.32344616-0.21108196j, -0.24629772+0.14464823j,
        -0.18616437+0.20046521j,  0.82895613-0.08502687j],
       [ 0.20690755-0.27451764j, -0.69735823-0.16845287j,
         0.5680897 +0.10766273j, -0.08200131-0.16156443j],
       [-0.29561328-0.3020349j , -0.10014621+0.23311884j,
        -0.24497241+0.67640124j, -0.46592275+0.14966605j]])

In [133]:
cirq.decompose(cirq.H(a))

[(cirq.Y**0.5).on(cirq.GridQubit(3, 4)),
 cirq.XPowGate(exponent=1.0, global_shift=-0.25).on(cirq.GridQubit(3, 4))]

In [147]:
class MyGate(cirq.Gate):
    def __init__(self):
        super(MyGate, self)

    def _num_qubits_(self):
        return 2

    def _unitary_(self):
        return np.array([[ 0.13015817+0.73287303j, -0.54007754+0.21833927j,
        -0.25825206+0.08133333j, -0.02023084+0.18133301j],
       [-0.32344616-0.21108196j, -0.24629772+0.14464823j,
        -0.18616437+0.20046521j,  0.82895613-0.08502687j],
       [ 0.20690755-0.27451764j, -0.69735823-0.16845287j,
         0.5680897 +0.10766273j, -0.08200131-0.16156443j],
       [-0.29561328-0.3020349j , -0.10014621+0.23311884j,
        -0.24497241+0.67640124j, -0.46592275+0.14966605j]])

    def _circuit_diagram_info_(self, args):
        return "G"
    
    def __str__(self):
        return "G"

In [152]:
cg = MyGate()
optest = cg.on(b,a)

In [153]:
cirq.decompose(optest)

[<__main__.MyGate object at 0x0108F410>.on(cirq.GridQubit(3, 5), cirq.GridQubit(3, 4))]

In [154]:
cirq.unitary(optest)

array([[ 0.13015817+0.73287303j, -0.54007754+0.21833927j,
        -0.25825206+0.08133333j, -0.02023084+0.18133301j],
       [-0.32344616-0.21108196j, -0.24629772+0.14464823j,
        -0.18616437+0.20046521j,  0.82895613-0.08502687j],
       [ 0.20690755-0.27451764j, -0.69735823-0.16845287j,
         0.5680897 +0.10766273j, -0.08200131-0.16156443j],
       [-0.29561328-0.3020349j , -0.10014621+0.23311884j,
        -0.24497241+0.67640124j, -0.46592275+0.14966605j]])

In [156]:
# cirq.decompose_once(optest)
# TypeError: object of type '<class 'cirq.ops.gate_operation.GateOperation'>' does have a _decompose_ method, but it returned NotImplemented or None.